In [52]:

#UTILITIES
import pandas as pd 
import numpy as np 
import datetime


from ast import literal_eval
import itertools
import time
import re

#Some Settings

pd.set_option('display.max_columns', 100)

RANDOM_STATE = len('I used to be an adventurer like you') #But I took an arrow to the knee.

In [303]:
#Data import

#Dataset gathered using steam API
raw_steam = pd.read_csv('steam_app_data.csv')
#Steam Spy API
raw_steamspy = pd.read_csv('steamspy_data.csv')


In [95]:
raw_steam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37726 entries, 0 to 37725
Data columns (total 39 columns):
type                       37630 non-null object
name                       37723 non-null object
steam_appid                37726 non-null int64
required_age               37630 non-null float64
is_free                    37630 non-null object
controller_support         7856 non-null object
dlc                        6470 non-null object
detailed_description       37609 non-null object
about_the_game             37608 non-null object
short_description          37609 non-null object
fullgame                   0 non-null float64
supported_languages        37607 non-null object
header_image               37630 non-null object
website                    23648 non-null object
pc_requirements            37630 non-null object
mac_requirements           37630 non-null object
linux_requirements         37630 non-null object
legal_notice               12746 non-null object
drm_notice    

In [96]:
raw_steam.head()

,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
0,game,Counter-Strike,10,0.0,False,NaN,NaN,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,NaN,"English<strong>*</strong>, French<strong>*</st...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'USD', 'initial': 999, 'final': 9...",[7],"[{'name': 'default', 'title': 'Buy Counter-Str...","{'windows': True, 'mac': True, 'linux': True}","{'score': 88, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 92459},NaN,"{'coming_soon': False, 'date': 'Nov 1, 2000'}","{'url': 'http://steamcommunity.com/app/10', 'e...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
1,game,Team Fortress Classic,20,0.0,False,NaN,NaN,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,NaN,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/20/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'USD', 'initial': 499, 'final': 4...",[29],"[{'name': 'default', 'title': 'Buy Team Fortre...","{'windows': True, 'mac': True, 'linux': True}",NaN,NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 3491},NaN,"{'coming_soon': False, 'date': 'Apr 1, 1999'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/20/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
2,game,Day of Defeat,30,0.0,False,NaN,NaN,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,NaN,"English, French, German, Italian, Spanish - Spain",https://steamcdn-a.akamaihd.net/steam/apps/30/...,http://www.dayofdefeat.com/,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'USD', 'initial': 499, 'final': 4...",[30],"[{'name': 'default', 'title': 'Buy Day of Defe...","{'windows': True, 'mac': True, 'linux': True}","{'score': 79, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 2634},NaN,"{'coming_soon': False, 'date': 'May 1, 2003'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/30/...,"{'ids': [], 'notes': None}"
3,game,Deathmatch Classic,40,0.0,False,NaN,NaN,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,NaN,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/40/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: 

In [97]:
metacritic_raw = raw_steam[['name', 'steam_appid', 'metacritic', 'reviews', 'recommendations']]

## Data Cleaning

In [98]:
null = raw_steam.isnull().sum()
null

type                          96
name                           3
steam_appid                    0
required_age                  96
is_free                       96
controller_support         29870
dlc                        31256
detailed_description         117
about_the_game               118
short_description            117
fullgame                   37726
supported_languages          119
header_image                  96
website                    14078
pc_requirements               96
mac_requirements              96
linux_requirements            96
legal_notice               24980
drm_notice                 37522
ext_user_account_notice    37000
developers                   201
publishers                    96
demos                      34927
price_overview              5702
packages                    5173
package_groups                96
platforms                     96
metacritic                 34374
reviews                    30731
categories                   854
genres    

There are severl columns missing more than half of the information. Since these wont help us we're going to drop each of these columns. 

In [99]:
#Set a threshold for dropping null values

threshold = raw_steam.shape[0] // 2
print('the threshold is:', threshold)

the threshold is: 18863


In [100]:
#Get a list of columns to drop
drop_col = raw_steam.columns[null > threshold]

In [101]:
print('Columns to drop: {}'.format(list(drop_col)))

Columns to drop: ['controller_support', 'dlc', 'fullgame', 'legal_notice', 'drm_notice', 'ext_user_account_notice', 'demos', 'metacritic', 'reviews', 'recommendations']


In [102]:
print('Rows to remove:', raw_steam[raw_steam['type'].isnull()].shape[0])

raw_steam[raw_steam['type'].isnull()].head()

Rows to remove: 96


,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
36,NaN,ValveTestApp852,852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
157,NaN,Star Trek: DAC - Demo,4330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
266,NaN,Puzzlegeddon,8740,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
274,NaN,Borderlands DLC: Claptrap's New Robot Revolution,8955,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
397,NaN,America's Army 3 Beta,13120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [103]:
#Check Value Counts
raw_steam['type'].value_counts(dropna=False)

game    37630
NaN        96
Name: type, dtype: int64

This matches the above so we're cleared to proceed

In [104]:
#Moving on to name column, check for missing game names

missing_names = raw_steam[(raw_steam['name'].isnull()) | (raw_steam['name'] == 'none')]
missing_names

,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
489,NaN,NaN,17760,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4928,game,none,339860,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,English<strong>*</strong><br><strong>*</strong...,https://steamcdn-a.akamaihd.net/steam/apps/339...,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],NaN,NaN,NaN,NaN,[''],NaN,NaN,NaN,[],"{'windows': True, 'mac': False, 'linux': False}",NaN,NaN,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '25', 'description': 'Adventure'}, {'i...",NaN,NaN,NaN,"{'total': 3, 'highlighted': [{'name': 'Master ...","{'coming_soon': False, 'date': 'Feb 27, 2015'}","{'url': '', 'email': ''}",NaN,"{'ids': [], 'notes': None}"
6789,game,none,385020,0.0,False,NaN,NaN,- discontinued - (please remove),- discontinued - (please remove),- discontinued - (please remove),NaN,"English, French, Italian, German, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/385...,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,NaN,NaN,NaN,['none'],[''],NaN,NaN,NaN,[],"{'windows': True, 'mac': True, 'linux': True}",NaN,NaN,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '4', 'description': 'Casual'}, {'id': ...",NaN,NaN,NaN,NaN,"{'coming_soon': False, 'date': 'Nov 4, 2015'}","{'url': '', 'email': ''}",NaN,"{'ids': [], 'notes': None}"
7239,game,NaN,396420,0.0,True,NaN,NaN,Spookeningは3Dの恐怖ゲームで、あなたは毎夜に死んでゴーストとして復活します。<b...,Spookeningは3Dの恐怖ゲームで、あなたは毎夜に死んでゴーストとして復活します。<b...,Spookeningは3Dの恐怖ゲームで、あなたは毎夜に死んでゴーストとして復活します。 村...,NaN,NaN,https://steamcdn-a.akamaihd.net/steam/apps/396...,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],NaN,NaN,NaN,NaN,[''],NaN,NaN,NaN,[],"{'windows': True, 'mac': False, 'linux': False}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'coming_soon': False, 'date': 'Nov 1, 2016'}","{'url': '', 'email': ''}",NaN,"{'ids': [], 'notes': None}"
7352,game,none,398970,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,English<strong>*</strong><br><strong>*</strong...,https://steamcdn-a.akamaihd.net/steam/apps/398...,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,NaN,NaN,NaN,['none'],['none'],"[{'appid': 516340, 'description': ''}]",NaN,NaN,[],"{'windows': True, 'mac': True, 'linux': True}",NaN,NaN,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '25', 'description': 'Adventure'}, {'i...",NaN,NaN,NaN,"{'total': 35, 'highlighted': [{'name': ""They'v...","{'coming_soon': False, 'date': 'Nov 5, 2015'}","{'url': '', 'email': ''}",NaN,"{'ids': [], 'notes': None}"
34988,game,NaN,1216780,0.0,False,NaN,NaN,&quot;Our Journeys 2 ~ A Collection of Visual ...,&quot;Our Journeys 2 ~ A Collection of Visual ...,&quot;Our Journeys 2 ~ A Collection of Visual ...,NaN,"English, French",https://steamcdn-a.akamaihd.net/steam/apps/121...,http://miaqc.ca/en,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],NaN,NaN,NaN,NaN,[''],NaN,NaN,NaN,[],"{'windows': True, 'mac': False, 'linux': False}",NaN,NaN,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '4', 'description': 'Casual'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://steamcdn...","[{'id': 256775407, 'name': 'Ixelda Shirina Tra...",NaN,"{'total': 20, 'highlighted': [{'name': 'Good E...","{'coming_soon': False, 'date': 'Mar 3, 2020'}","{'url': 'http://miaqc.ca/en', 'emai

Only six observations missing names (and a lot of other info) so we'll drop these too

In [105]:
duplicate_rows = raw_steam[raw_steam.duplicated()]

print('Dupicate rows to remove:', duplicate_rows.shape[0])

duplicate_rows.head()

Dupicate rows to remove: 16


,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
10,game,Counter-Strike,10,0.0,False,NaN,NaN,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,NaN,"English<strong>*</strong>, French<strong>*</st...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'USD', 'initial': 999, 'final': 9...",[7],"[{'name': 'default', 'title': 'Buy Counter-Str...","{'windows': True, 'mac': True, 'linux': True}","{'score': 88, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 92459},NaN,"{'coming_soon': False, 'date': 'Nov 1, 2000'}","{'url': 'http://steamcommunity.com/app/10', 'e...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
11,game,Team Fortress Classic,20,0.0,False,NaN,NaN,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,NaN,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/20/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'USD', 'initial': 499, 'final': 4...",[29],"[{'name': 'default', 'title': 'Buy Team Fortre...","{'windows': True, 'mac': True, 'linux': True}",NaN,NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 3491},NaN,"{'coming_soon': False, 'date': 'Apr 1, 1999'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/20/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
12,game,Day of Defeat,30,0.0,False,NaN,NaN,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,NaN,"English, French, German, Italian, Spanish - Spain",https://steamcdn-a.akamaihd.net/steam/apps/30/...,http://www.dayofdefeat.com/,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'USD', 'initial': 499, 'final': 4...",[30],"[{'name': 'default', 'title': 'Buy Day of Defe...","{'windows': True, 'mac': True, 'linux': True}","{'score': 79, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 2634},NaN,"{'coming_soon': False, 'date': 'May 1, 2003'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/30/...,"{'ids': [], 'notes': None}"
13,game,Deathmatch Classic,40,0.0,False,NaN,NaN,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,NaN,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/40/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minim

Now its time to treat the null values (be gone demons)

In [106]:
def drop_null_cols(df, thresh=0.5):
    '''Drop columns with null values over the established threshold'''
    cutoff = len(df) * thresh
    
    return df.dropna(thresh=cutoff, axis = 1)

def drop_name_type(df):
    '''Drop rows with null values and none for game type and name'''
    #remove rows where type col is blank
    df = df[df['type'].notnull()]
    #remove rows where name is not defined
    df = df[df['name'].notnull()]
    df = df[df['name'] != 'none']
    #remove type col
    df = df.drop('type', axis=1)
    
    return df

def process(df):
    '''Process the dataframe to clean and use functions we create'''
    
    #copy df to avoid unhappy accidents
    df = df.copy()
    
    #Remove duplicate rows
    df = df.drop_duplicates()
    
    #Remove col with more than 50% null val
    df = drop_null_cols(df)
    
    #Remove rows with with null vals for game and type then drop type col
    df = drop_name_type(df)
    
    return df



In [107]:
print(raw_steam.shape)
initial_df = process(raw_steam)
print(initial_df.shape)
initial_df.head()

(37726, 39)
(37609, 28)


,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,price_overview,packages,package_groups,platforms,categories,genres,screenshots,movies,achievements,release_date,support_info,background,content_descriptors
0,Counter-Strike,10,0.0,False,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,"English<strong>*</strong>, French<strong>*</st...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],"{'currency': 'USD', 'initial': 999, 'final': 9...",[7],"[{'name': 'default', 'title': 'Buy Counter-Str...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,NaN,"{'coming_soon': False, 'date': 'Nov 1, 2000'}","{'url': 'http://steamcommunity.com/app/10', 'e...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
1,Team Fortress Classic,20,0.0,False,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/20/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],"{'currency': 'USD', 'initial': 499, 'final': 4...",[29],"[{'name': 'default', 'title': 'Buy Team Fortre...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,NaN,"{'coming_soon': False, 'date': 'Apr 1, 1999'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/20/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
2,Day of Defeat,30,0.0,False,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,"English, French, German, Italian, Spanish - Spain",https://steamcdn-a.akamaihd.net/steam/apps/30/...,http://www.dayofdefeat.com/,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],"{'currency': 'USD', 'initial': 499, 'final': 4...",[30],"[{'name': 'default', 'title': 'Buy Day of Defe...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,NaN,"{'coming_soon': False, 'date': 'May 1, 2003'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/30/...,"{'ids': [], 'notes': None}"
3,Deathmatch Classic,40,0.0,False,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/40/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],"{'currency': 'USD', 'initial': 499, 'final': 4...",[31],"[{'name': 'default', 'title': 'Buy Deathmatch ...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,NaN,"{

# Age

The next column to look at is required_age. 

In [108]:
null_df = initial_df.isnull().sum()
print('empty age requirements:', null_df['required_age'])

empty age requirements: 0


In [109]:
initial_df['required_age'].value_counts(dropna=False).sort_index()

0.0     36584
1.0         1
3.0         3
5.0         1
6.0         4
7.0         3
10.0       22
11.0        1
12.0       25
13.0      146
14.0        3
15.0        6
16.0       52
17.0      583
18.0      172
20.0        1
21.0        2
Name: required_age, dtype: int64

The range of age is between 0 and 21.

Video games are a largely self regulated industry. In the US, ESRB puts ratings on games for recommended ages. In the rest of the world, PEGI puts ratings on games. ESRB uses letters (E, E10+, T, M) and PEGI uses numbers for ages (3, 7, 12, 16, 18). 

Binning by ages seeems more appropriate here so we'll use PEGI system.

https://www.esrb.org/

https://pegi.info/


In [110]:
initial_df[initial_df['required_age'] == 21]

,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,price_overview,packages,package_groups,platforms,categories,genres,screenshots,movies,achievements,release_date,support_info,background,content_descriptors
22758,Booty Calls,823550,21.0,True,Help mermaid princess Andriella fulfill her ta...,Help mermaid princess Andriella fulfill her ta...,Help mermaid princess Andriella fulfill her ta...,"English<strong>*</strong>, French, German, Spa...",https://steamcdn-a.akamaihd.net/steam/apps/823...,https://www.bootycallsgame.com/,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],['3X Entertainment Limited'],['3X Entertainment Limited'],NaN,NaN,[],"{'windows': True, 'mac': True, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '4', 'description': 'Casual'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://steamcdn...","[{'id': 256723441, 'name': 'Teaser', 'thumbnai...",NaN,"{'coming_soon': False, 'date': 'Sep 18, 2018'}","{'url': 'http://support.bootycallsgame.com', '...",https://steamcdn-a.akamaihd.net/steam/apps/823...,"{'ids': [1, 3, 5], 'notes': 'The content of Bo..."
35579,Hades Ultimate Fighting Ball,1240680,21.0,False,Hades Ultimate Fighting Ball is a Multiplayer ...,Hades Ultimate Fighting Ball is a Multiplayer ...,Hades Ultimate Fighting Ball is a multiplayer ...,"French<strong>*</strong>, English, Spanish - S...",https://steamcdn-a.akamaihd.net/steam/apps/124...,https://hades-game.com/,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,"['Mustieles Irvin', 'Atenza Nicolas']","['Mustieles Irvin', 'Atenza Nicolas']","{'currency': 'USD', 'initial': 599, 'final': 5...",[430581],"[{'name': 'default', 'title': 'Buy Hades Ultim...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://steamcdn...","[{'id': 256778899, 'name': 'Hades UFB shortvid...",NaN,"{'coming_soon': False, 'date': 'Mar 31, 2020'}","{'url': 'https://hades-game.com/', 'email': 's...",https://steamcdn-a.akamaihd.net/steam/apps/124...,"{'ids': [1, 2, 5], 'notes': 'Violence, coarse ..."


In [111]:
def pegi_age(df):
    '''Bin age requirments according to PEGI standards'''
    #PEGI Age Ratings 3, 7, 12, 16, 18
    
    bins = [-1, 0, 3, 7, 12, 16, 2000]
    labels = [0, 3, 7, 12, 16, 18]
    
    df['required_age'] = pd.cut(df['required_age'], bins = bins, labels =labels)
    
    return df

In [112]:
age_df = pegi_age(initial_df)


In [113]:
age_df['required_age'].value_counts().sort_index()

0     36584
3         4
7         8
12       48
16      207
18      758
Name: required_age, dtype: int64

## Price (to be cont...)

The next column is whether the game is free to play or not displayed in boolean. It makes sense to deal with this when we deal with the price overview in the future. But lets take a sneak peak.

In [114]:
age_df['is_free'].value_counts()

False    33344
True      4265
Name: is_free, dtype: int64

In [115]:
age_df['price_overview'].head()

0    {'currency': 'USD', 'initial': 999, 'final': 9...
1    {'currency': 'USD', 'initial': 499, 'final': 4...
2    {'currency': 'USD', 'initial': 499, 'final': 4...
3    {'currency': 'USD', 'initial': 499, 'final': 4...
4    {'currency': 'USD', 'initial': 499, 'final': 4...
Name: price_overview, dtype: object

is_free column is next but it makes sense to treat this with the cost column so we'll skip ahead to platforms

## Platforms Column

Platforms on steam is what kind of Operating System the game can be played (Windows, MAC OS, linux)

In [116]:
age_df['platforms'].head()

0    {'windows': True, 'mac': True, 'linux': True}
1    {'windows': True, 'mac': True, 'linux': True}
2    {'windows': True, 'mac': True, 'linux': True}
3    {'windows': True, 'mac': True, 'linux': True}
4    {'windows': True, 'mac': True, 'linux': True}
Name: platforms, dtype: object

This column has a dictionary structure with keys representing the operating system and values representing whether the game can be played on that os. 

In [117]:
null_df['platforms']

0

No null values to worry about in this column.

In [118]:
def clean_platforms(df):
    '''Split platforms column into seperate boolean columns for each platform'''
    #evaluate values in platforms column
    df = df.copy()
    
    def parse_platforms(x):
        
        d = literal_eval(x)
        
        return ';'.join(platform for platform in d.keys() if d[platform])
    
    df['platforms'] = df['platforms'].apply(parse_platforms)
    
    return df

In [119]:
platforms_df = clean_platforms(age_df)
platforms_df['platforms'].value_counts()

windows              26423
windows;mac;linux     5533
windows;mac           4648
windows;linux          997
mac                      6
mac;linux                1
linux                    1
Name: platforms, dtype: int64

Majority of games are only availabe on windows. With only one game available exclusively on linux. 

In [120]:
platforms_df.head()

,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,price_overview,packages,package_groups,platforms,categories,genres,screenshots,movies,achievements,release_date,support_info,background,content_descriptors
0,Counter-Strike,10,0,False,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,"English<strong>*</strong>, French<strong>*</st...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],"{'currency': 'USD', 'initial': 999, 'final': 9...",[7],"[{'name': 'default', 'title': 'Buy Counter-Str...",windows;mac;linux,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,NaN,"{'coming_soon': False, 'date': 'Nov 1, 2000'}","{'url': 'http://steamcommunity.com/app/10', 'e...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
1,Team Fortress Classic,20,0,False,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/20/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],"{'currency': 'USD', 'initial': 499, 'final': 4...",[29],"[{'name': 'default', 'title': 'Buy Team Fortre...",windows;mac;linux,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,NaN,"{'coming_soon': False, 'date': 'Apr 1, 1999'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/20/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
2,Day of Defeat,30,0,False,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,"English, French, German, Italian, Spanish - Spain",https://steamcdn-a.akamaihd.net/steam/apps/30/...,http://www.dayofdefeat.com/,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],"{'currency': 'USD', 'initial': 499, 'final': 4...",[30],"[{'name': 'default', 'title': 'Buy Day of Defe...",windows;mac;linux,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,NaN,"{'coming_soon': False, 'date': 'May 1, 2003'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/30/...,"{'ids': [], 'notes': None}"
3,Deathmatch Classic,40,0,False,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/40/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],"{'currency': 'USD', 'initial': 499, 'final': 4...",[31],"[{'name': 'default', 'title': 'Buy Deathmatch ...",windows;mac;linux,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,NaN,"{'coming_soon': False, 'date': 'Jun 1, 2001'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/40/...,"{'id

## Price (For real this time)

In [121]:
null_df['price_overview']

5599

Before writing this off as a loss, lets see what gaps we can fill with the is_free column

In [122]:
free_null_price_index = platforms_df[(platforms_df['is_free']) & (platforms_df['price_overview'].isnull())]

free_null_price_index.shape[0]

4211

In [123]:
not_free_null_price = platforms_df[(platforms_df['is_free'] == False) & (platforms_df['price_overview'].isnull())]

not_free_null_price

,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,price_overview,packages,package_groups,platforms,categories,genres,screenshots,movies,achievements,release_date,support_info,background,content_descriptors
73,The Ship: Single Player,2420,0,False,For PC gamers who enjoy multiplayer games with...,For PC gamers who enjoy multiplayer games with...,The Ship is a murder mystery alternative to tr...,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/242...,http://www.blazinggriffin.com/games/the-ship-m...,{'minimum': '<strong>Minimum:</strong> 1.8 GHz...,[],[],['Outerlight Ltd.'],['Blazing Griffin Ltd.'],NaN,[56669],"[{'name': 'default', 'title': 'Buy The Ship: S...",windows,"[{'id': 2, 'description': 'Single-player'}]","[{'id': '1', 'description': 'Action'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://steamcdn...","[{'id': 2035597, 'name': 'the Ship: Intro', '...",NaN,"{'coming_soon': False, 'date': 'Nov 20, 2006'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/242...,"{'ids': [], 'notes': None}"
85,RollerCoaster Tycoon® 3: Platinum,2700,0,False,Rollercoaster Tycoon 3 Platinum combines the e...,Rollercoaster Tycoon 3 Platinum combines the e...,Rollercoaster Tycoon 3 Platinum combines the e...,"English, French, Italian, German, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/270...,http://www.atari.com/rollercoastertycoon/us/in...,{'minimum': '<strong>Minimum: </strong><br>\t\...,"{'minimum': '<ul class=""bb_ul""><li><strong>OS:...",[],"['Frontier', 'Aspyr (Mac)']","['Atari', 'Aspyr (Mac)']",NaN,NaN,[],windows;mac,"[{'id': 2, 'description': 'Single-player'}]","[{'id': '28', 'description': 'Simulation'}, {'...","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,NaN,"{'coming_soon': False, 'date': 'Mar 12, 2008'}","{'url': 'http://www.atari.com/support/atari', ...",https://steamcdn-a.akamaihd.net/steam/apps/270...,"{'ids': [], 'notes': None}"
116,Hammer Heads Deluxe,3400,0,False,Pesky yard gnomes are invading your lawn! The ...,Pesky yard gnomes are invading your lawn! The ...,Pesky yard gnomes are invading your lawn! The ...,English,https://steamcdn-a.akamaihd.net/steam/apps/340...,NaN,{'minimum': '<strong>Minimum Requirements:</st...,[],[],"['PopCap Games, Inc.']","['PopCap Games, Inc.']",NaN,NaN,[],windows,"[{'id': 2, 'description': 'Single-player'}]","[{'id': '4', 'description': 'Casual'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,NaN,"{'coming_soon': False, 'date': 'Aug 30, 2006'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/340...,"{'ids': [], 'notes': None}"
120,Rocket Mania Deluxe,3440,0,False,Can you become master of pyrotechnics? Twist ...,Can you become master of pyrotechnics? Twist ...,Can you become master of pyrotechnics? Twist a...,English,https://steamcdn-a.akamaihd.net/steam/apps/344...,NaN,{'minimum': '<strong>Minimum Requirements:</st...,[],[],"['PopCap Games, Inc.']","['PopCap Games, Inc.']",NaN,NaN,[],windows,"[{'id': 2, 'description': 'Single-player'}]","[{'id': '4', 'description': 'Casual'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,NaN,"{'coming_soon': False, 'date': 'Aug 30, 2006'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/344...,"{'ids': [], 'notes': None}"
230,BioShock™,7670,18,False,<h1>Special Offer</h1><p>Buying BioShock™ also...,BioShock is a shooter unlike any you've ever p...,BioShock is a shooter unlike any you've ever p...,"English<strong>*</strong>, French<strong>*</st...",https://steamcdn-a.akamaihd.net/steam/apps/767...,http://www.BioShockGame.com,"{'minimum': '<h2 class=""bb_tag""><strong>Minimu...",{'minimum': 'Please See BioShock Remastered'},[],"['2K Boston', '2K Australia']",['2K'],NaN,"[451, 127633]","[{'name': 'default', 'title': 'Buy BioShock™',...",windows,"[{'

846 games in our dataframe do not have a price listed. This is odd so lets investigate a handful.

In [124]:
not_free_sample = not_free_null_price.sample(n=5, random_state=RANDOM_STATE)

https://store.steampowered.com/app

In [125]:
def print_steam_links(df):
    url= 'https://store.steampowered.com/app/'
    
    for i, row in df.iterrows():
        appid = row['steam_appid']
        name = row['name']
        
        print(name + ':', url + str(appid))
    

In [126]:
print_steam_links(not_free_sample)

Marvel: Ultimate Alliance 2: https://store.steampowered.com/app/433320
R.B.I. Baseball 16: https://store.steampowered.com/app/407690
IL-2 Sturmovik: Cliffs of Dover: https://store.steampowered.com/app/63950
VR Golf Online: https://store.steampowered.com/app/575540
Grandpa and the Zombies: https://store.steampowered.com/app/732390


* Eagle Island is a demo page

* Rain of Reflections was in development at time fo data collection

* Holobunnies does not have a price listed as it is in active development

* Norman's Great Illusion was in development at time of data collection 

* Viscera Cleanup Detail: Shadow Warrior Multiple editions available

For various reasons, the games do not have a price. We'll push off treating these for now as there might be other reasons they get dropped. 


In [131]:
platforms_df['price_overview'][5]

"{'currency': 'USD', 'initial': 499, 'final': 499, 'discount_percent': 0, 'initial_formatted': '', 'final_formatted': '$4.99'}"

In [132]:
def clean_price(df):
    df = df.copy()
    
    def parse_price(x):
        if x is not np.nan:
            return literal_eval(x)
        else:
            return {'currency': 'USD', 'initial': -1}
    #evaluate as dictionary and set to -1 if missing
    df['price_overview'] = df['price_overview'].apply(parse_price)
    
    # Create columns from currency and initial values
    df['currency'] = df['price_overview'].apply(lambda x: x['currency'])
    df['price']    = df['price_overview'].apply(lambda x: x['initial'])
    
    # Free games are free
    df.loc[df['is_free'], 'price'] = 0 
    
    return df
        

In [133]:
price_data = clean_price(platforms_df)[['name', 'currency', 'price']]

price_data.head()

,name,currency,price
0,Counter-Strike,USD,999
1,Team Fortress Classic,USD,499
2,Day of Defeat,USD,499
3,Deathmatch Classic,USD,499
4,Half-Life: Opposing Force,USD,499


In [134]:
price_data[price_data['currency'] != 'USD']

,name,currency,price
19286,Roguebreaker,RUB,3000


There is one game that is not listed in USD, since the number is so small we'll markfordelete

In [137]:
def clean_price(df):
    '''clean price_overview col into formatted price column'''
    df = df.copy()
    
    def parse_price(x):
        if x is not np.nan:
            return literal_eval(x)
        else:
            return{'currency': 'USD', 'initial': -1}
    # eval as dictionary and set to -1 if missing
    df['price_overview'] = df['price_overview'].apply(parse_price)
    
    # create columns from currency and initial values
    df['currency'] = df['price_overview'].apply(lambda x: x['currency'])
    df['price'] = df['price_overview'].apply(lambda x: x['initial'])
    
    # set price of free games to 0
    df.loc[df['is_free'], 'price'] = 0 
    
    #remove non-USD rows
    df = df[df['currency']== 'USD']
    
    # Change price to display in pounds
    df.loc[df['price'] > 0, 'price']/= 100
    
    #remove columns no longer needed
    df = df.drop(['is_free', 'currency', 'price_overview'], axis=1)
    
    return df

In [138]:
price_df = clean_price(platforms_df)
price_df[['name', 'price']].head()

,name,price
0,Counter-Strike,9.99
1,Team Fortress Classic,4.99
2,Day of Defeat,4.99
3,Deathmatch Classic,4.99
4,Half-Life: Opposing Force,4.99


## Packages

In [139]:
with pd.option_context('display.max_colwidth', 500):
    display(price_df[['steam_appid', 'packages', 'package_groups', 'price']].head())

,steam_appid,packages,package_groups,price
0,10,[7],"[{'name': 'default', 'title': 'Buy Counter-Strike', 'description': '', 'selection_text': 'Select a purchase option', 'save_text': '', 'display_type': 0, 'is_recurring_subscription': 'false', 'subs': [{'packageid': 7, 'percent_savings_text': ' ', 'percent_savings': 0, 'option_text': 'Counter-Strike: Condition Zero - $9.99', 'option_description': '', 'can_get_free_license': '0', 'is_free_license': False, 'price_in_cents_with_discount': 999}]}]",9.99
1,20,[29],"[{'name': 'default', 'title': 'Buy Team Fortress Classic', 'description': '', 'selection_text': 'Select a purchase option', 'save_text': '', 'display_type': 0, 'is_recurring_subscription': 'false', 'subs': [{'packageid': 29, 'percent_savings_text': ' ', 'percent_savings': 0, 'option_text': 'Team Fortress Classic - $4.99', 'option_description': '', 'can_get_free_license': '0', 'is_free_license': False, 'price_in_cents_with_discount': 499}]}]",4.99
2,30,[30],"[{'name': 'default', 'title': 'Buy Day of Defeat', 'description': '', 'selection_text': 'Select a purchase option', 'save_text': '', 'display_type': 0, 'is_recurring_subscription': 'false', 'subs': [{'packageid': 30, 'percent_savings_text': ' ', 'percent_savings': 0, 'option_text': 'Day of Defeat - $4.99', 'option_description': '', 'can_get_free_license': '0', 'is_free_license': False, 'price_in_cents_with_discount': 499}]}]",4.99
3,40,[31],"[{'name': 'default', 'title': 'Buy Deathmatch Classic', 'description': '', 'selection_text': 'Select a purchase option', 'save_text': '', 'display_type': 0, 'is_recurring_subscription': 'false', 'subs': [{'packageid': 31, 'percent_savings_text': ' ', 'percent_savings': 0, 'option_text': 'Deathmatch Classic - $4.99', 'option_description': '', 'can_get_free_license': '0', 'is_free_license': False, 'price_in_cents_with_discount': 499}]}]",4.99
4,50,[32],"[{'name': 'default', 'title': 'Buy Half-Life: Opposing Force', 'description': '', 'selection_text': 'Select a purchase option', 'save_text': '', 'display_type': 0, 'is_recurring_subscription': 'false', 'subs': [{'packageid': 32, 'percent_savings_text': ' ', 'percent_savings': 0, 'option_text': 'Opposing Force - $4.99', 'option_description': '', 'can_get_free_license': '0', 'is_free_license': False, 'price_in_cents_with_discount': 499}]}]",4.99


In [140]:
print(price_df[price_df['price']== -1].shape[0])

1388


In [141]:
print('Null counts:', price_df['package_groups'].isnull().sum())
print('Empty list counts:', price_df[price_df['package_groups']=='[]'].shape[0])

Null counts: 0
Empty list counts: 5341


In [142]:
missing_price_package = price_df[(price_df['price'] == -1) & (price_df['package_groups'] == '[]')]

print('Number of rows:', missing_price_package.shape[0], '\n')

print('First few rows:\n')
print_steam_links(missing_price_package[:5])

print('\nLast few rows:\n')
print_steam_links(missing_price_package[-10:-5])

Number of rows: 1342 

First few rows:

RollerCoaster Tycoon® 3: Platinum: https://store.steampowered.com/app/2700
Hammer Heads Deluxe: https://store.steampowered.com/app/3400
Rocket Mania Deluxe: https://store.steampowered.com/app/3440
Telltale Texas Hold ‘Em: https://store.steampowered.com/app/8330
Strong Bad's Cool Game for Attractive People: Season 1: https://store.steampowered.com/app/8340

Last few rows:

Tabletop Playground Beta: https://store.steampowered.com/app/1282050
Eternal Vortex - Beta: https://store.steampowered.com/app/1287980
Prophecy: https://store.steampowered.com/app/1289800
Project Océara: https://store.steampowered.com/app/1290820
梦中女孩: https://store.steampowered.com/app/1292790


In [143]:
def process_price(df):
    
    """Process price_overview column into formatted price column, and take care of package columns."""
    df = df.copy()
    def parse_price(x):
        if x is not np.nan:
            return literal_eval(x)
        else:
            return {'currency': 'USD', 'initial': -1}
    # evaluate as dictionary and set to -1 if missing
    df['price_overview'] = df['price_overview'].apply(parse_price)
    # create columns from currency and initial values
    df['currency'] = df['price_overview'].apply(lambda x: x['currency'])
    df['price'] = df['price_overview'].apply(lambda x: x['initial'])
    # set price of free games to 0
    df.loc[df['is_free'], 'price'] = 0
    # remove non-USD rows
    df = df[df['currency'] == 'USD']
    # remove rows where price is -1
    df = df[df['price'] != -1]
    # change price to display in pounds (can apply to all now -1 rows removed)
    df['price'] /= 100
    # remove columns no longer needed
    df = df.drop(['is_free', 'currency', 'price_overview', 'packages', 'package_groups'], axis=1)
    
    return df



In [144]:
price_df = process_price(platforms_df)
price_df.head()

,name,steam_appid,required_age,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,platforms,categories,genres,screenshots,movies,achievements,release_date,support_info,background,content_descriptors,price
0,Counter-Strike,10,0,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,"English<strong>*</strong>, French<strong>*</st...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],windows;mac;linux,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,NaN,"{'coming_soon': False, 'date': 'Nov 1, 2000'}","{'url': 'http://steamcommunity.com/app/10', 'e...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,"{'ids': [2, 5], 'notes': 'Includes intense vio...",9.99
1,Team Fortress Classic,20,0,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/20/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],windows;mac;linux,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,NaN,"{'coming_soon': False, 'date': 'Apr 1, 1999'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/20/...,"{'ids': [2, 5], 'notes': 'Includes intense vio...",4.99
2,Day of Defeat,30,0,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,"English, French, German, Italian, Spanish - Spain",https://steamcdn-a.akamaihd.net/steam/apps/30/...,http://www.dayofdefeat.com/,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],windows;mac;linux,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,NaN,"{'coming_soon': False, 'date': 'May 1, 2003'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/30/...,"{'ids': [], 'notes': None}",4.99
3,Deathmatch Classic,40,0,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/40/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],windows;mac;linux,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,NaN,"{'coming_soon': False, 'date': 'Jun 1, 2001'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/40/...,"{'ids': [], 'notes': None}",4.99
4,Half-Life: Opposing Force,50,0,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,"English, French, German, Korean",https://steamcdn-a.akamaihd.net/steam/apps/50/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Gearbox Software'],['Valve'],win

## Languages

In [145]:
price_df['supported_languages'].isnull().sum()

7

In [146]:
price_df[price_df['supported_languages'].isnull()]

,name,steam_appid,required_age,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,platforms,categories,genres,screenshots,movies,achievements,release_date,support_info,background,content_descriptors,price
3990,Defense Grid 2: A Matter of Endurance,312860,0,NaN,NaN,NaN,NaN,https://steamcdn-a.akamaihd.net/steam/apps/312...,NaN,[],[],[],NaN,[''],windows,NaN,NaN,NaN,NaN,NaN,"{'coming_soon': False, 'date': 'Sep 22, 2014'}","{'url': '', 'email': ''}",NaN,"{'ids': [], 'notes': None}",5.99
4875,Subsiege,338640,0,"<img src=""https://steamcdn-a.akamaihd.net/stea...","<img src=""https://steamcdn-a.akamaihd.net/stea...",Subsiege is an intense real-time tactic game w...,NaN,https://steamcdn-a.akamaihd.net/steam/apps/338...,http://subsiege-game.com/,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],['Icebird Studios'],['Icebird Studios'],windows,NaN,NaN,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...","[{'id': 256729398, 'name': 'Release Trailer', ...",NaN,"{'coming_soon': False, 'date': 'Sep 7, 2018'}","{'url': 'http://subsiege-game.com/', 'email': ...",https://steamcdn-a.akamaihd.net/steam/apps/338...,"{'ids': [], 'notes': None}",19.99
14406,MARS VR(全球使命VR),596560,0,1.\t4K level audio-visual experience <br />\r\...,1.\t4K level audio-visual experience <br />\r\...,Welcome to 《Mars VR》. This is an immersive fir...,NaN,https://steamcdn-a.akamaihd.net/steam/apps/596...,http://qqsm.zygames.com/,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,"['Ying Pei Digital Technology Shanghai Co., Li...","['SHANGHAI ZHENYOU TECHNOLOGY CO.,LTD']",windows,"[{'id': 2, 'description': 'Single-player'}]","[{'id': '73', 'description': 'Violent'}, {'id'...","[{'id': 0, 'path_thumbnail': 'https://steamcdn...","[{'id': 256681371, 'name': 'marsvr', 'thumbnai...",NaN,"{'coming_soon': False, 'date': 'Apr 5, 2017'}","{'url': 'http://www.zygames.com/contact', 'ema...",https://steamcdn-a.akamaihd.net/steam/apps/596...,"{'ids': [], 'notes': None}",2.99
16243,Numberline 2,654970,0,NumberLine 2 is the continuation of the popula...,NumberLine 2 is the continuation of the popula...,NumberLine 2 is the continuation of the popula...,NaN,https://steamcdn-a.akamaihd.net/steam/apps/654...,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],"['V34D4R', 'Egor Magurin']",['Indovers Studio'],windows,"[{'id': 2, 'description': 'Single-player'}]","[{'id': '4', 'description': 'Casual'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://steamcdn...","[{'id': 256687192, 'name': 'Numberline 2 Trail...","{'total': 60, 'highlighted': [{'name': '1st le...","{'coming_soon': False, 'date': 'Jul 14, 2017'}","{'url': '', 'email': 'radaew.zhenya@yandex.ru'}",https://steamcdn-a.akamaihd.net/steam/apps/654...,"{'ids': [], 'notes': None}",1.99
22006,It Could Have Been Me,803840,0,"<h2 class=""bb_tag"">It Could Have Been Me</h2><...","<h2 class=""bb_tag"">It Could Have Been Me</h2><...",A journey from Afganistan to Sweden. Experienc...,NaN,https://steamcdn-a.akamaihd.net/steam/apps/803...,https://www.facebook.com/1866040833678994/vide...,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],"['Åsa Egnér vr-designer', 'Alexander Cobleigh ...",['Kreativitetsbanken'],windows,"[{'id': 2, 'description': 'Single-player'}]","[{'id': '23', 'description': 'Indie'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...","[{'id': 256749583, 'name': 'it could have been...",NaN,"{'coming_soon': False, 'date': 'May 19, 2019'}","{'url': 'http://www.kreativitetsbanken.nu/', '...",https://steamcdn-a.akamaihd.net/steam/apps/803...,"{'ids': [], 'notes': None}",2.99
27010,SNUSE 221,948070,0,<strong> Hey. My name is *&amp;#!$.<br>Today I...,<strong> Hey. My name is *&amp;#!$.<br>Today I...,Hey. My name is *&amp;#!$. Today I will tell y...,NaN,https://steamcdn-a.akamaihd.net/steam/apps/948

In [147]:
print(price_df['supported_languages'][0])
price_df['supported_languages'].value_counts().head(10)

English<strong>*</strong>, French<strong>*</strong>, German<strong>*</strong>, Italian<strong>*</strong>, Spanish - Spain<strong>*</strong>, Simplified Chinese<strong>*</strong>, Traditional Chinese<strong>*</strong>, Korean<strong>*</strong><br><strong>*</strong>languages with full audio support


English                                                                                                        10499
English<strong>*</strong><br><strong>*</strong>languages with full audio support                                9377
English, Russian                                                                                                 768
English, Simplified Chinese                                                                                      418
English, Japanese                                                                                                365
Simplified Chinese                                                                                               308
Simplified Chinese<strong>*</strong><br><strong>*</strong>languages with full audio support                      243
English<strong>*</strong>, Russian<strong>*</strong><br><strong>*</strong>languages with full audio support      223
English, French, Italian, German, Spanish - Spain               

In [148]:
def process_language(df):
    """Process supported_languages column into a boolean 'is english' column."""
    df = df.copy()
    # drop rows with missing language data
    df = df.dropna(subset=['supported_languages'])
    df['english'] = df['supported_languages'].apply(lambda x: 1 if 'english' in x.lower() else 0)
    df = df.drop('supported_languages', axis=1)
    return df
language_df = process_language(price_df)
language_df[['name', 'english']].head()

,name,english
0,Counter-Strike,1
1,Team Fortress Classic,1
2,Day of Defeat,1
3,Deathmatch Classic,1
4,Half-Life: Opposing Force,1


In [149]:
language_df['english'].value_counts()

1    35249
0      964
Name: english, dtype: int64

## Developers and Publishers

In [150]:
print('Developers null counts:', language_df['developers'].isnull().sum())
print('Developers empty list counts:', language_df[language_df['developers'] == "['']"].shape[0])
print('\nPublishers null counts:', language_df['publishers'].isnull().sum())
print('Publishers empty list counts:', language_df[language_df['publishers'] == "['']"].shape[0])

Developers null counts: 67
Developers empty list counts: 0

Publishers null counts: 0
Publishers empty list counts: 236


In [151]:
no_dev = language_df[language_df['developers'].isnull()]

print('Total games missing developer:', no_dev.shape[0], '\n')

print_steam_links(no_dev.sample(n=5, random_state= RANDOM_STATE))

no_pub = language_df[language_df['publishers'] == "['']"]

print('\nTotal games missing publisher:', no_pub.shape[0], '\n')

print_steam_links(no_pub.sample(n=5, random_state= RANDOM_STATE))

no_dev_pub = language_df[(language_df['developers'].isnull()) & (language_df['publishers'] == "['']")]

print('\nTotal games missing developer and publisher:', no_dev_pub.shape[0], '\n')
print_steam_links(no_dev_pub.sample(n=5, random_state=RANDOM_STATE))

Total games missing developer: 67 

2 Planets Fire and Ice: https://store.steampowered.com/app/657110
Artemis Spaceship Bridge Simulator: https://store.steampowered.com/app/247350
Atlantic Quest Solitaire: https://store.steampowered.com/app/379260
Survival Tycoon: https://store.steampowered.com/app/667920
Path to the Sky: https://store.steampowered.com/app/370390

Total games missing publisher: 236 

RIP - Trilogy™: https://store.steampowered.com/app/2540
Forest Warrior: https://store.steampowered.com/app/378590
Crystal Cosmos: https://store.steampowered.com/app/496590
BattleStick: https://store.steampowered.com/app/394380
Tetradecagon: https://store.steampowered.com/app/487380

Total games missing developer and publisher: 27 

CRACKPOT DESPOT: TRUMP WARFARE: https://store.steampowered.com/app/910290
SAMOLIOTIK: https://store.steampowered.com/app/449680
Sign Here: _________: https://store.steampowered.com/app/845330
Fester Mudd: Curse of the Gold - Episode 1: https://store.steampowered

In [152]:
def clean_devs_pubs(df):
    '''clean developers and publisher columns'''
    
    #remove rows with missing data
    df = df[(df['developers'].notnull()) & (df['publishers'] != "['']")].copy()
    df = df[~(df['developers'].str.contains(';')) & ~(df['publishers'].str.contains(';'))]
    df = df[(df['publishers'] != "['NA']") & (df['publishers'] != "['N/A']")]
    
    #create list 
    df['developer'] = df['developers'].apply(lambda x: ';'.join(literal_eval(x)))
    df['publisher'] = df['publishers'].apply(lambda x: ';'.join(literal_eval(x)))
    
    df = df.drop(['developers', 'publishers'], axis = 1)
    
    return df

    

In [153]:
dev_pub_df = clean_devs_pubs(language_df)
dev_pub_df[['name', 'steam_appid', 'developer', 'publisher']].head()

,name,steam_appid,developer,publisher
0,Counter-Strike,10,Valve,Valve
1,Team Fortress Classic,20,Valve,Valve
2,Day of Defeat,30,Valve,Valve
3,Deathmatch Classic,40,Valve,Valve
4,Half-Life: Opposing Force,50,Gearbox Software,Valve


In [154]:
dev_pub_df.to_csv('checkpoint_df.csv')

## Categories

In [155]:
def process_categories_and_genres(df):
    df = df.copy()
    df = df[(df['categories'].notnull()) & (df['genres'].notnull())]
    for col in ['categories', 'genres']:
        df[col] = df[col].apply(lambda x: ';'.join(item['description'] for item in literal_eval(x)))
    return df


cat_gen_df = process_categories_and_genres(dev_pub_df)
cat_gen_df[['steam_appid', 'categories', 'genres']].head()
    

,steam_appid,categories,genres
0,10,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action
1,20,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action
2,30,Multi-player;Valve Anti-Cheat enabled,Action
3,40,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action
4,50,Single-player;Multi-player;Valve Anti-Cheat en...,Action


In [156]:
## Achivements

In [157]:
def process_achievements_and_descriptors(df):
    """Parse as total number of achievements."""
    df = df.copy()
    df = df.drop('content_descriptors', axis=1)
    def parse_achievements(x):
        if x is np.nan:
            # missing data, assume has no achievements
            return 0
        else:
            # else has data, so can extract and return number under total
            return literal_eval(x)['total']
    df['achievements'] = df['achievements'].apply(parse_achievements)
    return df


achiev_df = process_achievements_and_descriptors(cat_gen_df)
achiev_df['achievements'].value_counts().head()

0     15525
10      885
12      782
20      732
15      632
Name: achievements, dtype: int64

In [159]:
def process(df):
    """Process data set. Will eventually contain calls to all functions we write."""
    # Copy the input dataframe to avoid accidentally modifying original data
    df = df.copy()
    # Remove duplicate rows - all appids should be unique
    df = df.drop_duplicates()
    # Remove collumns with more than 50% null values
    df = drop_null_cols(df)
    # Process columns
    df = drop_name_type(df)
    df = pegi_age(df)
    df = clean_platforms(df)
    df = clean_price(df)
    df = process_language(df)
    df = clean_devs_pubs(df)
    df = process_categories_and_genres(df)
    df = process_achievements_and_descriptors(df)
    return df

partially_clean = process(raw_steam)
partially_clean.head()

,name,steam_appid,required_age,detailed_description,about_the_game,short_description,header_image,website,pc_requirements,mac_requirements,linux_requirements,packages,package_groups,platforms,categories,genres,screenshots,movies,achievements,release_date,support_info,background,price,english,developer,publisher
0,Counter-Strike,10,0,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,https://steamcdn-a.akamaihd.net/steam/apps/10/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",[7],"[{'name': 'default', 'title': 'Buy Counter-Str...",windows;mac;linux,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,0,"{'coming_soon': False, 'date': 'Nov 1, 2000'}","{'url': 'http://steamcommunity.com/app/10', 'e...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,9.99,1,Valve,Valve
1,Team Fortress Classic,20,0,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,https://steamcdn-a.akamaihd.net/steam/apps/20/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",[29],"[{'name': 'default', 'title': 'Buy Team Fortre...",windows;mac;linux,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,0,"{'coming_soon': False, 'date': 'Apr 1, 1999'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/20/...,4.99,1,Valve,Valve
2,Day of Defeat,30,0,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,https://steamcdn-a.akamaihd.net/steam/apps/30/...,http://www.dayofdefeat.com/,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",[30],"[{'name': 'default', 'title': 'Buy Day of Defe...",windows;mac;linux,Multi-player;Valve Anti-Cheat enabled,Action,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,0,"{'coming_soon': False, 'date': 'May 1, 2003'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/30/...,4.99,1,Valve,Valve
3,Deathmatch Classic,40,0,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,https://steamcdn-a.akamaihd.net/steam/apps/40/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",[31],"[{'name': 'default', 'title': 'Buy Deathmatch ...",windows;mac;linux,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,0,"{'coming_soon': False, 'date': 'Jun 1, 2001'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/40/...,4.99,1,Valve,Valve
4,Half-Life: Opposing Force,50,0,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,https://steamcdn-a.akamaihd.net/steam/apps/50/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",[32],"[{'name': 'default', 'title': 'Buy Half-Life: ...",windows;mac;linux,Single-player;Multi-player;Valve Anti-Cheat en...,Action,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,0,"{'coming_soon': False, 'date': 'Nov 1, 1999'}","{'url': 'https://help.steampowered.com', 'emai...",https://steamcdn-a.akamaihd.net/steam/apps/50/...,4.99,1,Gearbox Software,Valve


In [160]:
partially_clean[['detailed_description', 'about_the_game', 'short_description']].isnull().sum()

detailed_description    2
about_the_game          3
short_description       2
dtype: int64

In [162]:
partially_clean[partially_clean['detailed_description'].isnull()].head()

,name,steam_appid,required_age,detailed_description,about_the_game,short_description,header_image,website,pc_requirements,mac_requirements,linux_requirements,packages,package_groups,platforms,categories,genres,screenshots,movies,achievements,release_date,support_info,background,price,english,developer,publisher
29835,GoK,1037110,0,NaN,NaN,The game is in recreation mode ! New news in 5...,https://steamcdn-a.akamaihd.net/steam/apps/103...,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,NaN,[],windows,Single-player,Action;Adventure;Indie;RPG;Early Access,NaN,"[{'id': 256743476, 'name': 'GoK', 'thumbnail':...",0,"{'coming_soon': False, 'date': 'Mar 26, 2019'}","{'url': '', 'email': 'gamespaceenergy@gmail.com'}",NaN,-1.0,1,GSE,GSE
32017,Creatures,1109790,0,NaN,NaN,Sandbox nature simulation. Create your creatur...,https://steamcdn-a.akamaihd.net/steam/apps/110...,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],NaN,[],windows,Single-player,Indie;Simulation;Strategy;Early Access,NaN,NaN,0,"{'coming_soon': False, 'date': 'Sep 16, 2019'}","{'url': '', 'email': 'developer.simyc@gmail.com'}",https://steamcdn-a.akamaihd.net/steam/apps/110...,-1.0,1,Simyc Games,Simyc Games


In [164]:
partially_clean[partially_clean['detailed_description'].str.len()<=30]

,name,steam_appid,required_age,detailed_description,about_the_game,short_description,header_image,website,pc_requirements,mac_requirements,linux_requirements,packages,package_groups,platforms,categories,genres,screenshots,movies,achievements,release_date,support_info,background,price,english,developer,publisher
3093,The Sands Whisper Your Name,282960,0,Under Development,Under Development,Under Development,https://steamcdn-a.akamaihd.net/steam/apps/282...,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],NaN,[],windows;mac;linux,Single-player,Indie;Early Access,NaN,NaN,0,"{'coming_soon': False, 'date': 'Nov 27, 2014'}","{'url': '', 'email': ''}",NaN,-1.00,1,NexyMedia,NexyMedia
3393,Uriel's Chasm,292630,0,Religious de-programming tool,Religious de-programming tool,Religious de-programming tool,https://steamcdn-a.akamaihd.net/steam/apps/292...,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],[41233],"[{'name': 'default', 'title': ""Buy Uriel's Cha...",windows,Single-player;Steam Trading Cards,Adventure;Casual;Indie,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...","[{'id': 2034394, 'name': ""Uriel's Chasm"", 'thu...",0,"{'coming_soon': False, 'date': 'Sep 12, 2014'}","{'url': '', 'email': 'rail_slave@hotmail.co.uk'}",https://steamcdn-a.akamaihd.net/steam/apps/292...,4.99,1,Rail Slave Games,KPL
4909,Factions: Origins of Malu,339530,0,Updating Coming,Updating Coming,Update Coming,https://steamcdn-a.akamaihd.net/steam/apps/339...,http://www.originsofmalu.com,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,NaN,[],windows;mac;linux,Multi-player;Cross-Platform Multiplayer,Action;Adventure;Casual;Indie;Strategy;Early A...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...","[{'id': 256661443, 'name': 'Lore Short', 'thum...",0,"{'coming_soon': False, 'date': 'Mar 23, 2015'}","{'url': 'www.originsofmalu.com', 'email': 'sup...",https://steamcdn-a.akamaihd.net/steam/apps/339...,-1.00,1,Burning Dog Media,Burning Dog Media
5519,Stephen's Sausage Roll,353540,0,A simple 3d puzzle game.,A simple 3d puzzle game.,A simple 3d puzzle game.,https://steamcdn-a.akamaihd.net/steam/apps/353...,http://www.stephenssausageroll.com,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,[61935],"[{'name': 'default', 'title': ""Buy Stephen's S...",windows;mac;linux,Single-player;Full controller support;Captions...,Indie,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...","[{'id': 2037511, 'name': 'Main Trailer', 'thum...",0,"{'coming_soon': False, 'date': 'Apr 17, 2016'}","{'url': '', 'email': 'analytic+ssrsupport@gmai...",https://steamcdn-a.akamaihd.net/steam/apps/353...,29.99,1,increpare games,increpare games
18978,KEKW,723360,0,KEKW,KEKW,KEKW,https://steamcdn-a.akamaihd.net/steam/apps/723...,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,NaN,[],windows,Single-player;Multi-player;PvP;Online PvP;Shar...,Action,NaN,NaN,0,"{'coming_soon': True, 'date': ''}","{'url': '', 'email': 'playkekw@gmail.com'}",https://steamcdn-a.akamaihd.net/steam/apps/723...,-1.00,1,Pog Media,Pog Media
19310,JumpSky,731910,0,It has been closed,It has been closed,It has been closed,https://steamcdn-a.akamaihd.net/steam/apps/731...,NaN,[],{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,NaN,[],windows;mac,Single-player,Casual,NaN,NaN,0,"{'coming_soon': False, 'date': 'Nov 1, 2017'}","{'url': '', 'email': ''}",NaN,-1.00,1,none,none
21788,God Test,797660,0,God Test,God Test,God Test,https://steamcdn-a.akamaihd.net/steam/apps/797...,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],NaN,[],windows,Multi-player;MMO;PvP;Online PvP;Co-op;Online C...,Action;Massively Multiplayer;RPG;Strategy,NaN,NaN,0,"{'coming_soo

In [165]:
def export_data(df, filename):
    """Export dataframe to csv file, filename prepended with 'steam_'.
    filename : str without file extension
    """
    filepath =  filename + '.csv'
    df.to_csv(filepath, index=False)
    print_name = filename.replace('_', ' ')
    print("Exported {} to '{}'".format(print_name, filepath))

In [166]:
def process_descriptions(df, export=False):
    """Export descriptions to external csv file then remove these columns."""
    # remove rows with missing description data
    df = df[df['detailed_description'].notnull()].copy()
    # remove rows with unusually small description
    df = df[df['detailed_description'].str.len() > 20]
    # by default we don't export, useful if calling function later
    if export:
        # create dataframe of description columns
        description_data = df[['steam_appid', 'detailed_description', 'about_the_game', 'short_description']]
        export_data(description_data, filename='description_data')
    # drop description columns from main dataframe
    df = df.drop(['detailed_description', 'about_the_game', 'short_description'], axis=1)
    return df



desc_df = process_descriptions(partially_clean, export=True)

Exported description data to 'description_data.csv'


In [167]:
def process_media(df, export=False):
    """Remove media columns from dataframe, optionally exporting them to csv first."""
    df = df[df['screenshots'].notnull()].copy()
    if export:
        media_data = df[['steam_appid', 'header_image', 'screenshots', 'background', 'movies']]
        export_data(media_data, 'media_data')
    df = df.drop(['header_image', 'screenshots', 'background', 'movies'], axis=1)
    return df
media_df = process_media(desc_df, export=True)

Exported media data to 'media_data.csv'


In [168]:
print('Before removing data:\n')
achiev_df.info(verbose=False, memory_usage="deep")

print('\nData with descriptions and media removed:\n')
media_df.info(verbose=False, memory_usage="deep")

Before removing data:

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35222 entries, 0 to 37725
Columns: 24 entries, name to publisher
dtypes: category(1), float64(1), int64(3), object(19)
memory usage: 378.0 MB

Data with descriptions and media removed:

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36428 entries, 0 to 37725
Columns: 19 entries, name to publisher
dtypes: category(1), float64(1), int64(3), object(14)
memory usage: 90.8 MB


In [169]:
def process_info(df, export=False):
    """Drop support information from dataframe, optionally exporting beforehand."""
    if export:
        
        support_info = df[['steam_appid', 'website', 'support_info']].copy()
        support_info['support_info'] = support_info['support_info'].apply(lambda x: literal_eval(x))
        support_info['support_url'] = support_info['support_info'].apply(lambda x: x['url'])
        support_info['support_email'] = support_info['support_info'].apply(lambda x: x['email'])
        support_info = support_info.drop('support_info', axis=1)
        
        # only keep rows with at least one piece of information
        support_info = support_info[(support_info['website'].notnull()) | (support_info['support_url'] != '') | (support_info['support_email'] != '')]
        export_data(support_info, 'support_info')
    df = df.drop(['website', 'support_info'], axis=1)
    return df


info_df = process_info(media_df, export=True)

Exported support info to 'support_info.csv'


In [170]:
def process_requirements(df, export=False):
    if export:
        requirements = df[['steam_appid', 'pc_requirements', 'mac_requirements', 'linux_requirements']].copy()
        # remove rows with missing pc requirements
        requirements = requirements[requirements['pc_requirements'] != '[]']
        requirements['requirements_clean'] = (requirements['pc_requirements']
                                                  .str.replace(r'\\[rtn]', '')
                                                  .str.replace(r'<[pbr]{1,2}>', ' ')
                                                  .str.replace(r'<[\/"=\w\s]+>', '')
                                             )
        requirements['requirements_clean'] = requirements['requirements_clean'].apply(lambda x: literal_eval(x))
        # split out minimum and recommended into separate columns
        requirements['minimum'] = requirements['requirements_clean'].apply(lambda x: x['minimum'].replace('Minimum:', '').strip() if 'minimum' in x.keys() else np.nan)
        requirements['recommended'] = requirements['requirements_clean'].apply(lambda x: x['recommended'].replace('Recommended:', '').strip() if 'recommended' in x.keys() else np.nan)
        requirements = requirements.drop('requirements_clean', axis=1)
        export_data(requirements, 'requirements_data')
    df = df.drop(['pc_requirements', 'mac_requirements', 'linux_requirements'], axis=1)
    return df


reqs_df = process_requirements(info_df, export=True)

Exported requirements data to 'requirements_data.csv'


In [215]:
def process_release_date(df):
    df = df.copy()
    
    def eval_date(x):
        x = literal_eval(x)
        if x['coming_soon']:
            return '' # return blank string so can drop missing at end
        else:
            return x['date']
    
    df['release_date'] = df['release_date'].apply(eval_date)
    

    df['release_date'] = pd.to_datetime(df['release_date'])
    
    df = df[df['release_date'].notnull()]
    
    return df

In [218]:
def process(df):
    """Process data set. Will eventually contain calls to all functions we write."""
    
    # Copy the input dataframe to avoid accidentally modifying original data
    df = df.copy()
    
    # Remove duplicate rows - all appids should be unique
    df = df.drop_duplicates()
    
    # Remove collumns with more than 50% null values
    df = drop_null_cols(df)
    
    # Process columns
    df = drop_name_type(df)
    df = pegi_age(df)
    df = clean_platforms(df)
    df = process_price(df)
    df = process_language(df)
    df = clean_devs_pubs(df)
    df = process_categories_and_genres(df)
    df = process_achievements_and_descriptors(df)
    df = process_release_date(df)
    
    
    # Process columns which export data
    df = process_descriptions(df, export=True)
    df = process_media(df, export=True)
    df = process_info(df, export=True)
    df = process_requirements(df, export=True)
    
    return df

steam_data = process(raw_steam)
steam_data.head()

Exported description data to 'description_data.csv'
Exported media data to 'media_data.csv'
Exported support info to 'support_info.csv'
Exported requirements data to 'requirements_data.csv'


,name,steam_appid,required_age,platforms,categories,genres,achievements,release_date,price,english,developer,publisher
0,Counter-Strike,10,0,windows;mac;linux,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,0,2000-11-01,9.99,1,Valve,Valve
1,Team Fortress Classic,20,0,windows;mac;linux,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,0,1999-04-01,4.99,1,Valve,Valve
2,Day of Defeat,30,0,windows;mac;linux,Multi-player;Valve Anti-Cheat enabled,Action,0,2003-05-01,4.99,1,Valve,Valve
3,Deathmatch Classic,40,0,windows;mac;linux,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,0,2001-06-01,4.99,1,Valve,Valve
4,Half-Life: Opposing Force,50,0,windows;mac;linux,Single-player;Multi-player;Valve Anti-Cheat en...,Action,0,1999-11-01,4.99,1,Gearbox Software,Valve


In [220]:
steam_data.isnull().sum()

name            0
steam_appid     0
required_age    0
platforms       0
categories      0
genres          0
achievements    0
release_date    0
price           0
english         0
developer       0
publisher       0
dtype: int64

In [221]:
raw_steam.info(verbose=False, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37726 entries, 0 to 37725
Columns: 39 entries, type to content_descriptors
dtypes: float64(2), int64(1), object(36)
memory usage: 510.0 MB


In [222]:
steam_data.info(verbose = False, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35174 entries, 0 to 37725
Columns: 12 entries, name to publisher
dtypes: category(1), datetime64[ns](1), float64(1), int64(3), object(6)
memory usage: 17.9 MB


In [224]:
steam_data[steam_data['release_date'] > '2020-08-1']

,name,steam_appid,required_age,platforms,categories,genres,achievements,release_date,price,english,developer,publisher
5622,The Survivor,356030,0,windows;linux,Single-player;Steam Trading Cards;Partial Cont...,Action;Adventure;Indie;Early Access,0,2020-08-03,1.99,1,Starship Studio,Starship Studio
6706,Empyrion - Galactic Survival,383120,0,windows,Single-player;Multi-player;PvP;Online PvP;Co-o...,Adventure;Indie;Simulation;Strategy,0,2020-08-05,19.99,1,Eleon Game Studios,Eleon Game Studios
9567,Massive,461400,0,windows;linux,Single-player;Multi-player;PvP;Online PvP;Co-o...,Action;Adventure;Indie,3,2020-08-05,9.99,1,Rootify,Rootify
12714,Approaching Infinity,551620,0,windows,Single-player;Steam Achievements,Adventure;Indie;Simulation;Strategy;Early Access,55,2020-08-05,10.99,1,IBOLOGY LLC,IBOLOGY LLC
16311,UnderMine,656350,0,windows;mac;linux,Single-player;Steam Achievements;Full controll...,Action;Adventure;Indie;RPG,96,2020-08-06,19.99,1,Thorium,Thorium
...,...,...,...,...,...,...,...,...,...,...,...,...
37720,Monster Killer,1376940,0,windows,Single-player;Steam Achievements;Steam Leaderb...,Casual;Indie,4,2020-08-05,0.99,1,Laush Dmitriy Sergeevich,Laush Studio
37721,Don't Look,1377140,0,windows;mac;linux,Single-player,Adventure;Indie,3,2020-08-07,4.99,1,Diego Aguilar,Diego Aguilar
37722,Train Train Train,1377900,0,windows,Single-player;Steam Achievements,Casual;Indie;Simulation;Strategy,18,2020-08-07,4.99,1,heinn,heinn
37723,The Citadel,1378290,0,windows,Single-player,Action;Casual;Indie,0,2020-08-05,14.99,1,doekuramori,doekuramori


In [225]:
steam_data.to_csv('clean_steam_data', index=False)

# Steam Spy

In [228]:
raw_steamspy.head()

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
0,10,Counter-Strike,Valve,Valve,NaN,159745,4173,0,"10,000,000 .. 20,000,000",21211,1237,172,81,999.0,999.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,15194,"{'Action': 5314, 'FPS': 4718, 'Multiplayer': 3..."
1,20,Team Fortress Classic,Valve,Valve,NaN,4335,771,0,"2,000,000 .. 5,000,000",798,0,11,0,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,123,"{'Action': 731, 'FPS': 297, 'Multiplayer': 251..."
2,30,Day of Defeat,Valve,Valve,NaN,4325,482,0,"5,000,000 .. 10,000,000",894,7,18,7,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Spain",Action,140,"{'FPS': 777, 'World War II': 242, 'Multiplayer..."
3,40,Deathmatch Classic,Valve,Valve,NaN,1592,345,0,"5,000,000 .. 10,000,000",1200,0,8,0,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,5,"{'Action': 624, 'FPS': 135, 'Classic': 103, 'M..."
4,50,Half-Life: Opposing Force,Gearbox Software,Valve,NaN,8907,477,0,"5,000,000 .. 10,000,000",1316,391,281,391,499.0,499.0,0.0,"English, French, German, Korean",Action,104,"{'FPS': 869, 'Action': 310, 'Classic': 237, 'S..."


In [229]:
raw_steamspy.isnull().sum()

appid                  0
name                   9
developer            187
publisher            201
score_rank         37671
positive               0
negative               0
userscore              0
owners                 0
average_forever        0
average_2weeks         0
median_forever         0
median_2weeks          0
price                 29
initialprice          22
discount              22
languages             88
genre                223
ccu                    0
tags                   0
dtype: int64

In [230]:
rss = raw_steamspy

In [231]:
rss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37717 entries, 0 to 37716
Data columns (total 20 columns):
appid              37717 non-null int64
name               37708 non-null object
developer          37530 non-null object
publisher          37516 non-null object
score_rank         46 non-null float64
positive           37717 non-null int64
negative           37717 non-null int64
userscore          37717 non-null int64
owners             37717 non-null object
average_forever    37717 non-null int64
average_2weeks     37717 non-null int64
median_forever     37717 non-null int64
median_2weeks      37717 non-null int64
price              37688 non-null float64
initialprice       37695 non-null float64
discount           37695 non-null float64
languages          37629 non-null object
genre              37494 non-null object
ccu                37717 non-null int64
tags               37717 non-null object
dtypes: float64(4), int64(9), object(7)
memory usage: 5.8+ MB


In [234]:
rss['score_rank'].value_counts().head()

98.0     17
100.0    12
99.0      8
97.0      8
96.0      1
Name: score_rank, dtype: int64

In [235]:
drop_cols = [
    'score_rank', # too many missing values
    'userscore', # too little variance (most have 0)
    'genre', 'developer', 'publisher', 'price', 'initialprice', 'discount', # provided by Steam data
    'average_2weeks', 'median_2weeks', 'ccu' # not interested in temporally specific columns
]

In [239]:
tags = rss['tags']
parsed_tags = tags.apply(lambda x: literal_eval(x))
cols =set(itertools.chain(*parsed_tags))

In [240]:
def parse_tags(x):
    x = literal_eval(x)
    if isinstance(x, dict):
        return x
    elif isinstance(x, list):
        return {}
    else:
        raise TypeError('Something other than dict or list found')

parsed_tags = tags.apply(parse_tags)
tag_data = pd.DataFrame()

for col in sorted(cols):
    # standardise column names
    col_name = col.lower().replace(' ', '_').replace('-', '_').replace("'", "")
    # check if column in row's dictionary of tags and return that value if it is, or 0 if it isn't
    tag_data[col_name] = parsed_tags.apply(lambda x: x[col] if col in x.keys() else 0)

    
tag_data.head()

,1980s,1990s,2.5d,2d,2d_fighter,2d_platformer,360_video,3d,3d_fighter,3d_platformer,3d_vision,4_player_local,4x,6dof,atv,abstract,action,action_rpg,action_rts,action_roguelike,action_adventure,addictive,adventure,agriculture,aliens,alternate_history,ambient,america,animation_&_modeling,anime,arcade,archery,arena_shooter,artificial_intelligence,assassin,asymmetric_vr,asynchronous_multiplayer,atmospheric,audio_production,auto_battler,automation,automobile_sim,bmx,base_building,baseball,based_on_a_novel,basketball,batman,battle_royale,beat_em_up,...,time_management,time_manipulation,time_travel,top_down,top_down_shooter,touch_friendly,tower_defense,trackir,trading,trading_card_game,traditional_roguelike,trains,transhumanism,transportation,trivia,turn_based,turn_based_combat,turn_based_strategy,turn_based_tactics,tutorial,twin_stick_shooter,typing,underground,underwater,unforgiving,utilities,vr,vr_only,vampire,vehicular_combat,video_production,villain_protagonist,violent,visual_novel,voice_control,voxel,walking_simulator,war,wargame,warhammer_40k,web_publishing,well_written,werewolves,western,word_game,world_war_i,world_war_ii,wrestling,zombies,e_sports
0,247,1151,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5314,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,220,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1144
1,0,121,0,0,0,0,0,0,0,0,0,0,0,0,0,0,731,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,38,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,154,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,144,0,0,0,0,0,0,0,13,242,0,0,0
3,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,624,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,128,0,0,0,0,0,0,0,0,0,0,0,0,0,0,310,0,0,0,0,0,109,0,163,0,0,0,0,0,0,0,0,0,0,0,0,100,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [241]:
def parse_tags(x):
    x = literal_eval(x)
    if isinstance(x, dict):
        return ';'.join(list(x.keys())[:3])
    else:
        return np.nan
tags.apply(parse_tags).head()

0          Action;FPS;Multiplayer
1          Action;FPS;Multiplayer
2    FPS;World War II;Multiplayer
3              Action;FPS;Classic
4              FPS;Action;Classic
Name: tags, dtype: object

In [244]:
def process_tags(df, export=False):
    if export: 
        tag_data = df[['appid', 'tags']].copy()
        def parse_export_tags(x):
            x = literal_eval(x)
            if isinstance(x, dict):
                return x
            elif isinstance(x, list):
                return {}
            else:
                raise TypeError('Something other than dict or list found')
        tag_data['tags'] = tag_data['tags'].apply(parse_export_tags)
        cols = set(itertools.chain(*tag_data['tags']))
        for col in sorted(cols):
            col_name = col.lower().replace(' ', '_').replace('-', '_').replace("'", "")
            tag_data[col_name] = tag_data['tags'].apply(lambda x: x[col] if col in x.keys() else 0)
        tag_data = tag_data.drop('tags', axis=1)
        tag_data.to_csv('steamspy_tag_data.csv', index=False)
        print("Exported tag data to 'steamspy_tag_data.csv'")
    def parse_tags(x):
        x = literal_eval(x)
        if isinstance(x, dict):
            return ';'.join(list(x.keys())[:3])
        else:
            return np.nan
    df['tags'] = df['tags'].apply(parse_tags)
    # rows with null tags seem to be superseded by newer release, so remove (e.g. dead island)
    df = df[df['tags'].notnull()]
    return df

def process(df):
    df = df.copy()
    # handle missing values
    df = df[(df['name'].notnull()) & (df['name'] != 'none')]
    df = df[df['developer'].notnull()]
    df = df[df['languages'].notnull()]
    df = df[df['price'].notnull()]
    # remove unwanted columns
    df = df.drop([
        'genre', 'developer', 'publisher', 'score_rank', 'userscore', 'average_2weeks',
        'median_2weeks', 'price', 'initialprice', 'discount', 'ccu'
    ], axis=1)
    # keep top tags, exporting full tag data to file
    df = process_tags(df, export=True)
    # reformat owners column
    df['owners'] = df['owners'].str.replace(',', '').str.replace(' .. ', '-')
    return df


steamspy_data = process(rss)
steamspy_data.head()

Exported tag data to 'steamspy_tag_data.csv'


,appid,name,positive,negative,owners,average_forever,median_forever,languages,tags
0,10,Counter-Strike,159745,4173,10000000-20000000,21211,172,"English, French, German, Italian, Spanish - Sp...",Action;FPS;Multiplayer
1,20,Team Fortress Classic,4335,771,2000000-5000000,798,11,"English, French, German, Italian, Spanish - Sp...",Action;FPS;Multiplayer
2,30,Day of Defeat,4325,482,5000000-10000000,894,18,"English, French, German, Italian, Spanish - Spain",FPS;World War II;Multiplayer
3,40,Deathmatch Classic,1592,345,5000000-10000000,1200,8,"English, French, German, Italian, Spanish - Sp...",Action;FPS;Classic
4,50,Half-Life: Opposing Force,8907,477,5000000-10000000,1316,281,"English, French, German, Korean",FPS;Action;Classic


In [246]:
steamspy_data.isnull().sum()

appid              0
name               0
positive           0
negative           0
owners             0
average_forever    0
median_forever     0
languages          0
tags               0
dtype: int64

In [248]:
steamspy_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37178 entries, 0 to 37716
Data columns (total 9 columns):
appid              37178 non-null int64
name               37178 non-null object
positive           37178 non-null int64
negative           37178 non-null int64
owners             37178 non-null object
average_forever    37178 non-null int64
median_forever     37178 non-null int64
languages          37178 non-null object
tags               37178 non-null object
dtypes: int64(5), object(4)
memory usage: 2.8+ MB


In [247]:
steamspy_data.to_csv('steamspy_clean.csv', index=False)

In [251]:
merged = steam_data.merge(steamspy_data, left_on='steam_appid', right_on='appid', suffixes=('','_steamspy'))

merged.head()                          
                          

,name,steam_appid,required_age,platforms,categories,genres,achievements,release_date,price,english,developer,publisher,appid,name_steamspy,positive,negative,owners,average_forever,median_forever,languages,tags
0,Counter-Strike,10,0,windows;mac;linux,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,0,2000-11-01,9.99,1,Valve,Valve,10,Counter-Strike,159745,4173,10000000-20000000,21211,172,"English, French, German, Italian, Spanish - Sp...",Action;FPS;Multiplayer
1,Team Fortress Classic,20,0,windows;mac;linux,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,0,1999-04-01,4.99,1,Valve,Valve,20,Team Fortress Classic,4335,771,2000000-5000000,798,11,"English, French, German, Italian, Spanish - Sp...",Action;FPS;Multiplayer
2,Day of Defeat,30,0,windows;mac;linux,Multi-player;Valve Anti-Cheat enabled,Action,0,2003-05-01,4.99,1,Valve,Valve,30,Day of Defeat,4325,482,5000000-10000000,894,18,"English, French, German, Italian, Spanish - Spain",FPS;World War II;Multiplayer
3,Deathmatch Classic,40,0,windows;mac;linux,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,0,2001-06-01,4.99,1,Valve,Valve,40,Deathmatch Classic,1592,345,5000000-10000000,1200,8,"English, French, German, Italian, Spanish - Sp...",Action;FPS;Classic
4,Half-Life: Opposing Force,50,0,windows;mac;linux,Single-player;Multi-player;Valve Anti-Cheat en...,Action,0,1999-11-01,4.99,1,Gearbox Software,Valve,50,Half-Life: Opposing Force,8907,477,5000000-10000000,1316,281,"English, French, German, Korean",FPS;Action;Classic


In [252]:
# remove overlapping columns
steam_clean = merged.drop(['name_steamspy', 'languages', 'steam_appid'], axis=1)
# reindex to reorder columns
steam_clean = steam_clean[[
    'appid',
    'name',
    'release_date',
    'english',
    'developer',
    'publisher',
    'platforms',
    'required_age',
    'categories',
    'genres',
    'tags',
    'achievements',
    'positive',
    'negative',
    'average_forever',
    'median_forever',
    'owners',
    'price'
]]
steam_clean = steam_clean.rename({
    'tags': 'steamspy_tags',
    'positive': 'positive_ratings',
    'negative': 'negative_ratings',
    'average_forever': 'average_playtime',
    'median_forever': 'median_playtime'
}, axis=1)
steam_clean.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,Action;FPS;Multiplayer,0,159745,4173,21211,172,10000000-20000000,9.99
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,Action;FPS;Multiplayer,0,4335,771,798,11,2000000-5000000,4.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,4325,482,894,18,5000000-10000000,4.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,Action;FPS;Classic,0,1592,345,1200,8,5000000-10000000,4.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Classic,0,8907,477,1316,281,5000000-10000000,4.99


In [253]:
#Export clean master

steam_clean.to_csv('steam_clean.csv', index = False)

In [254]:
df = pd.read_csv('steam_clean.csv')

In [255]:
df.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,Action;FPS;Multiplayer,0,159745,4173,21211,172,10000000-20000000,9.99
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,Action;FPS;Multiplayer,0,4335,771,798,11,2000000-5000000,4.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,4325,482,894,18,5000000-10000000,4.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,Action;FPS;Classic,0,1592,345,1200,8,5000000-10000000,4.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Classic,0,8907,477,1316,281,5000000-10000000,4.99


In [256]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35005 entries, 0 to 35004
Data columns (total 18 columns):
appid               35005 non-null int64
name                35005 non-null object
release_date        35005 non-null object
english             35005 non-null int64
developer           35005 non-null object
publisher           35004 non-null object
platforms           35005 non-null object
required_age        35005 non-null int64
categories          35005 non-null object
genres              35005 non-null object
steamspy_tags       35005 non-null object
achievements        35005 non-null int64
positive_ratings    35005 non-null int64
negative_ratings    35005 non-null int64
average_playtime    35005 non-null int64
median_playtime     35005 non-null int64
owners              35005 non-null object
price               35005 non-null float64
dtypes: float64(1), int64(8), object(9)
memory usage: 4.8+ MB


In [257]:
metacritic_raw.head()

,name,steam_appid,metacritic,reviews,recommendations
0,Counter-Strike,10,"{'score': 88, 'url': 'https://www.metacritic.c...",NaN,{'total': 92459}
1,Team Fortress Classic,20,NaN,NaN,{'total': 3491}
2,Day of Defeat,30,"{'score': 79, 'url': 'https://www.metacritic.c...",NaN,{'total': 2634}
3,Deathmatch Classic,40,NaN,NaN,{'total': 1222}
4,Half-Life: Opposing Force,50,NaN,NaN,{'total': 7156}


In [276]:
df = metacritic_raw

In [277]:
df = df[df['metacritic'].notna()]

In [278]:
df.head()

,name,steam_appid,metacritic,reviews,recommendations
0,Counter-Strike,10,"{'score': 88, 'url': 'https://www.metacritic.c...",NaN,{'total': 92459}
2,Day of Defeat,30,"{'score': 79, 'url': 'https://www.metacritic.c...",NaN,{'total': 2634}
6,Half-Life,70,"{'score': 96, 'url': 'https://www.metacritic.c...",NaN,{'total': 37530}
7,Counter-Strike: Condition Zero,80,"{'score': 65, 'url': 'https://www.metacritic.c...",NaN,{'total': 11945}
8,Half-Life: Blue Shift,130,"{'score': 71, 'url': 'https://www.metacritic.c...",NaN,{'total': 5466}


In [279]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3352 entries, 0 to 36574
Data columns (total 5 columns):
name               3352 non-null object
steam_appid        3352 non-null int64
metacritic         3352 non-null object
reviews            1716 non-null object
recommendations    2757 non-null object
dtypes: int64(1), object(4)
memory usage: 157.1+ KB


In [280]:
df['metacritic'][7]

"{'score': 65, 'url': 'https://www.metacritic.com/game/pc/counter-strike-condition-zero?ftag=MCD-06-10aaa1f'}"

In [281]:
def clean_critic(df):
    df = df.copy()
    
    def parse_critic(x):
        if x is not np.nan:
            return literal_eval(x)
        else:
            return{'score': -1, 'url': 'google.com'}
    
    df['metacritic'] = df['metacritic'].apply(parse_critic)
    
    #create columns for score and url
    
    df['meta_score'] = df['metacritic'].apply(lambda x: x['score'])
    df['meta_url'] = df['metacritic'].apply(lambda x: x['url'])
    
    return df

In [282]:
def clean_rec(df):
    df = df.copy()
    
    def parse_rec(x):
        if x is not np.nan:
            return literal_eval(x)
        else:
            return{'total': -1}
    
    df['recommendations'] = df['recommendations'].apply(parse_rec)
    
    #create columns for score and url
    
    df['total_recommendations'] = df['recommendations'].apply(lambda x: x['total'])
    
    
    return df

In [288]:
mc = clean_critic(df)

In [289]:
mc = clean_rec(mc)

In [290]:
mc.drop(['metacritic', 'name', 'reviews', 'recommendations', 'meta_url'], axis = 1, inplace=True)

In [291]:
mc.head()

,steam_appid,meta_score,total_recommendations
0,10,88,92459
2,30,79,2634
6,70,96,37530
7,80,65,11945
8,130,71,5466


In [292]:
mc.isnull().sum()

steam_appid              0
meta_score               0
total_recommendations    0
dtype: int64

In [293]:
mc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3352 entries, 0 to 36574
Data columns (total 3 columns):
steam_appid              3352 non-null int64
meta_score               3352 non-null int64
total_recommendations    3352 non-null int64
dtypes: int64(3)
memory usage: 104.8 KB


In [297]:
metacritic_merged = steam_clean.merge(mc, left_on='appid', right_on='steam_appid', suffixes=('', '_mc'))


In [298]:
metacritic_merged.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,steam_appid,meta_score,total_recommendations
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,Action;FPS;Multiplayer,0,159745,4173,21211,172,10000000-20000000,9.99,10,88,92459
1,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,Action;FPS;Multiplayer,0,159745,4173,21211,172,10000000-20000000,9.99,10,88,92459
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,4325,482,894,18,5000000-10000000,4.99,30,79,2634
3,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,4325,482,894,18,5000000-10000000,4.99,30,79,2634
4,70,Half-Life,1998-11-08,1,Valve,Valve,windows;mac;linux,0,Single-player;Multi-player;PvP;Online PvP;Stea...,Action,FPS;Classic;Action,0,48358,1819,2632,184,5000000-10000000,9.99,70,96,37530


In [307]:
# remove overlapping columns
metacritic_clean = metacritic_merged.drop(['steam_appid'], axis=1)
#drop duplicates
metacritic_clean = metacritic_clean.drop_duplicates(subset=['appid'])
# reindex to reorder columns
metacritic_clean = metacritic_clean[[
    'appid',
    'name',
    'release_date',
    'english',
    'developer',
    'publisher',
    'platforms',
    'required_age',
    'categories',
    'genres',
    'steamspy_tags',
    'achievements',
    'positive_ratings',
    'negative_ratings',
    'meta_score',
    'total_recommendations',
    'average_playtime',
    'median_playtime',
    'owners',
    'price'
]]

metacritic_clean.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,meta_score,total_recommendations,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,Action;FPS;Multiplayer,0,159745,4173,88,92459,21211,172,10000000-20000000,9.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,4325,482,79,2634,894,18,5000000-10000000,4.99
4,70,Half-Life,1998-11-08,1,Valve,Valve,windows;mac;linux,0,Single-player;Multi-player;PvP;Online PvP;Stea...,Action,FPS;Classic;Action,0,48358,1819,96,37530,2632,184,5000000-10000000,9.99
6,80,Counter-Strike: Condition Zero,2004-03-01,1,Valve,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,Action;FPS;Shooter,0,15679,1606,65,11945,1820,23,5000000-10000000,9.99
8,130,Half-Life: Blue Shift,2001-06-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Remote Play Together,Action,FPS;Action;Sci-fi,0,6439,672,71,5466,1317,191,5000000-10000000,4.99


In [308]:
metacritic_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3174 entries, 0 to 3182
Data columns (total 20 columns):
appid                    3174 non-null int64
name                     3174 non-null object
release_date             3174 non-null datetime64[ns]
english                  3174 non-null int64
developer                3174 non-null object
publisher                3174 non-null object
platforms                3174 non-null object
required_age             3174 non-null category
categories               3174 non-null object
genres                   3174 non-null object
steamspy_tags            3174 non-null object
achievements             3174 non-null int64
positive_ratings         3174 non-null int64
negative_ratings         3174 non-null int64
meta_score               3174 non-null int64
total_recommendations    3174 non-null int64
average_playtime         3174 non-null int64
median_playtime          3174 non-null int64
owners                   3174 non-null object
price             

In [309]:
metacritic_clean.to_csv('metacritic_clean.csv', index = False)

In [304]:
steam_df.head()


,metascore,name,console,userscore,release_date,app_id,review_score,review_percent_score
0,69,.hack//G.U. Last Recode,PC,6.5,2017-11-03,525480.0,8.0,87.0
1,59,007: NightFire,PC,6.0,2002-11-28,NaN,NaN,NaN
2,70,007: Quantum of Solace,PC,6.1,2008-11-04,NaN,NaN,NaN
3,73,0RBITALIS,PC,7.3,2015-05-28,278440.0,6.0,71.0
4,72,10 Second Ninja,PC,7.4,2014-03-05,271670.0,8.0,85.0


In [312]:
steam_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3790 entries, 0 to 3789
Data columns (total 8 columns):
metascore               3790 non-null int64
name                    3790 non-null object
console                 3790 non-null object
userscore               3790 non-null float64
release_date            3790 non-null object
app_id                  2324 non-null float64
review_score            2304 non-null float64
review_percent_score    2304 non-null float64
dtypes: float64(4), int64(1), object(3)
memory usage: 237.0+ KB
